<a href="https://colab.research.google.com/github/Amirsaj/Automated_Statistics/blob/main/automateStat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
# Load the Excel file
file_path = 'data.xlsx'
data = pd.read_excel(file_path)

FileNotFoundError: [Errno 2] No such file or directory: 'data.xlsx'

In [ ]:
columns_present = data.columns
numerical_columns = ['asa_rating_c', 'BMI', 'Number Mandible Sites Fx','age']
missing_columns = [col for col in numerical_columns if col not in columns_present]
column_data_types = data[numerical_columns].dtypes if not missing_columns else None
missing_columns, column_data_types

(['age'], None)

In [ ]:
from datetime import datetime

# Calculate age from birth_date
today = datetime.today()
data['age'] = data['birth_date'].apply(lambda x: today.year - x.year - ((today.month, today.day) < (x.month, x.day)))

data.drop('birth_date', axis=1, inplace=True)


In [ ]:
import pandas as pd

# Define a DataFrame with 5 columns: 'columnname', 'numberNo', 'percentNO', 'numberYes', 'percentYes'
resDF = pd.DataFrame(columns=['columnname', 'numberNo', 'percentNO', 'numberYes', 'percentYes'])


In [ ]:
import pandas as pd

# Data lists as described
catcolumns = data.columns
infokol = []
colname = []
resultcol = 'revision required (secondary outcome)'
for j in catcolumns:
    if j in ["asa_rating_c", "BMI", "Number Mandible Sites Fx",'SurgicalTime'] or j in numerical_columns:
      continue
    print(j)
    # Initial entry for the column itself with blank data (could adjust if needed)
    colname.append(j)
    infokol.append(["", "", "", ""])

    # Crosstab calculations
    ctdf = pd.crosstab(data[j], data[resultcol], normalize='index')
    ctdfn = pd.crosstab(data[j], data[resultcol])

    for i in ctdf.index:
        colname.append(i)  # Adding category value to the column names
        info = [
            ctdfn.loc[i, 'no'] if 'no' in ctdfn.columns else 0,
            ctdf.loc[i, 'no'] if 'no' in ctdf.columns else 0,
            ctdfn.loc[i, 'yes'] if 'yes' in ctdfn.columns else 0,
            ctdf.loc[i, 'yes'] if 'yes' in ctdf.columns else 0
        ]
        infokol.append(info)

# Creating DataFrame
resDF = pd.DataFrame(infokol, columns=['numberNo', 'percentNO', 'numberYes', 'percentYes'])
resDF['columnname'] = colname  # Assigning column names to a specific column if that was the intent

# Display the DataFrame
print(resDF.head())


sex
race
hispanic_c
ethnicity
state_name
zip
mallampati_score
primary_surgeons
tobacco
EtOH
Diabetes
mechanism
symphysis/parasymphysis
body
angle/ramus
condyle
Fracture complexity
other CMF injury
MMF
plating
approach
Complication (primary outcome)
Malocclusion
Plate Failure
Infection/hematoma
hardware exposure
non-union
no data/outside
other
revision required (secondary outcome)
revision surgery type
  numberNo percentNO numberYes percentYes                        columnname
0                                                                       sex
1       77  0.895349         9   0.104651                            Female
2      185  0.889423        23   0.110577                              Male
3                                                                      race
4        1       1.0         0        0.0  American Indian or Alaska Native


In [ ]:
# resDF.to_csv('crosstab_results.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

def calculate_stats(exposed_yes, exposed_no, control_yes, control_no):
    # Check if there are no cases in either exposed or control group
    if exposed_yes == 0 and control_yes == 0:
        return ["-", "-", "-", "-"]

    # Initialize variables to handle exceptions
    odds_ratio, ci_lower, ci_upper, chi2, p_value = "-", "-", "-", "-", "-"

    # Calculate odds ratio, handling division by zero
    try:
        odds_ratio = (exposed_yes * control_no) / (exposed_no * control_yes)
    except ZeroDivisionError:
        odds_ratio = "-"

    # Calculate chi-squared test and p-value
    try:
        table = np.array([[exposed_yes, exposed_no], [control_yes, control_no]])
        chi2, p_value, _, _ = stats.chi2_contingency(table)
    except:
        chi2, p_value = "-", "-"

    # Calculate confidence interval for the odds ratio
    if isinstance(odds_ratio, float):
        try:
            se_of_log_odds_ratio = np.sqrt((1/exposed_yes) + (1/exposed_no) + (1/control_yes) + (1/control_no))
            ci_lower = np.exp(np.log(odds_ratio) - 1.96 * se_of_log_odds_ratio)
            ci_upper = np.exp(np.log(odds_ratio) + 1.96 * se_of_log_odds_ratio)
        except:
            ci_lower, ci_upper = "-", "-"

    # Return the results
    return [odds_ratio if odds_ratio != "-" else odds_ratio,
            (ci_lower if ci_lower != "-" else "-", ci_upper if ci_upper != "-" else "-"),
            chi2, p_value]


In [ ]:
data[resultcol].value_counts()

revision required (secondary outcome)
no     262
yes     32
Name: count, dtype: int64

In [ ]:
total_control = data[resultcol].value_counts()['no']
total_exposed =  data[resultcol].value_counts()['yes']

In [ ]:
info = []
for i in range(len(resDF)):
    if resDF['numberYes'][i] == "":  # Check if numberYes is zero, after coercion to numeric
        info.append([0, 0, 0, 0])
    else:
        # print(resDF['numberYes'][i], total_exposed - resDF['numberYes'][i],
        #                          resDF['numberNo'][i], total_control - resDF['numberNo'][i])
        result = calculate_stats(resDF['numberYes'][i], total_exposed - resDF['numberYes'][i],
                                 resDF['numberNo'][i], total_control - resDF['numberNo'][i])
        info.append(result)

<ipython-input-9-250c0a72e3f1>:29: RuntimeWarning: divide by zero encountered in scalar divide
  se_of_log_odds_ratio = np.sqrt((1/exposed_yes) + (1/exposed_no) + (1/control_yes) + (1/control_no))
<ipython-input-9-250c0a72e3f1>:30: RuntimeWarning: divide by zero encountered in log
  ci_lower = np.exp(np.log(odds_ratio) - 1.96 * se_of_log_odds_ratio)
<ipython-input-9-250c0a72e3f1>:31: RuntimeWarning: divide by zero encountered in log
  ci_upper = np.exp(np.log(odds_ratio) + 1.96 * se_of_log_odds_ratio)
<ipython-input-9-250c0a72e3f1>:31: RuntimeWarning: invalid value encountered in scalar add
  ci_upper = np.exp(np.log(odds_ratio) + 1.96 * se_of_log_odds_ratio)
<ipython-input-9-250c0a72e3f1>:15: RuntimeWarning: divide by zero encountered in scalar divide
  odds_ratio = (exposed_yes * control_no) / (exposed_no * control_yes)
<ipython-input-9-250c0a72e3f1>:30: RuntimeWarning: invalid value encountered in scalar subtract
  ci_lower = np.exp(np.log(odds_ratio) - 1.96 * se_of_log_odds_ratio)


In [ ]:
new_data = pd.DataFrame(info, columns=["oddsratio", "CI", "chi2", "p_value"])

# Assuming your existing DataFrame is named df
# Append the new data

# Display the updated DataFrame

In [ ]:
for i in new_data.columns:
  resDF[i] = new_data[i]

In [ ]:
resDF.to_csv(resultcol+'categoricall.csv', index=False)

In [ ]:
numerical_columns = ['asa_rating_c', 'BMI', 'Number Mandible Sites Fx','age']


In [ ]:
import pandas as pd
from scipy import stats
import statsmodels.api as sm

def analyze_data(df, x_col, y_col):
    df.dropna(subset=[x_col, y_col], inplace=True)

    # Checking if columns exist in the dataframe
    if x_col not in df or y_col not in df:
        return "Column names are incorrect or do not exist in the dataframe."

    results = {}

    # Convert y_col to categorical if it's not numeric
    if df[y_col].dtype == 'object':
        df[y_col] = pd.Categorical(df[y_col]).codes
        results['info'] = f"{y_col} converted to numeric codes."

    # 1. Averages for x_col and y_col
    results['average_x'] = df[x_col].mean()
    results['average_y'] = df[y_col].mean()

    # 2. 0.25 and 0.75 Quantiles for x_col and y_col
    results['0.25_quantile_x'] = df[x_col].quantile(0.25)
    results['0.75_quantile_x'] = df[x_col].quantile(0.75)
    results['0.25_quantile_y'] = df[y_col].quantile(0.25)
    results['0.75_quantile_y'] = df[y_col].quantile(0.75)

    # 3. T-test between x_col and y_col
    t_test_result = stats.ttest_ind(df[x_col], df[y_col], equal_var=False)
    results['t_test_statistic'] = t_test_result.statistic
    results['t_test_p_value'] = t_test_result.pvalue

    # 4. Logistic Regression
    # Adding a constant term for intercept
    df['intercept'] = 1
    logit_model = sm.Logit(df[y_col], df[['intercept', x_col]])
    try:
        logit_result = logit_model.fit(disp=0)  # disp=0 suppresses fitting output
        results['logit_coefficients'] = logit_result.params
        results['logit_pvalues'] = logit_result.pvalues
    except Exception as e:
        results['logit_error'] = str(e)

    return results


In [ ]:
filtered_columns = [col for col in list(resDF.columns) if col != 'columnname']
irani = ['columnname']
for i in filtered_columns:
    irani.append(i)
resDF = resDF.reindex(columns=irani)

In [ ]:
layer1 = ["",'average_x',
'0.25_quantile_x',
'0.75_quantile_x',
't_test_p_value',
'logit_coefficients',
'',"",""]
layer = []
for i in numerical_columns:
  numdict = analyze_data(data,i,resultcol)
  layer.append([i,
      numdict['average_x'],
        numdict['0.25_quantile_x'],
        numdict['0.75_quantile_x'],
        numdict['t_test_p_value'],
        numdict['logit_coefficients'],"","",""])

In [ ]:
pd.DataFrame(layer,columns=layer1 ).to_csv(resultcol+"Numerical.csv")

AttributeError: 'DataFrame' object has no attribute 'append'